# Обедающие философы
 N мудрецов и N палочек для суши (или вилочек). Для еды нужно ухватить два столовых прибора.
 Для устранения блокировки (Deadlock) используется мехнизм мьютексов (Lock).
 Для завершения модели есть флаг running

In [ ]:
# -*- coding: utf-8 -*-
import threading, random, time

class Filosoff(threading.Thread):
    running = True

    def __init__(self, xname, forkOne, forkTwo):
        #threading.Thread.__init__(self)
        super().__init__()
        self.name = xname
        self.forkOnLeft = forkOne
        self.forkOnRight = forkTwo
        self.f_eat_count =0 

    def run(self):
        while self.running:
            #  Мудрец думает (на самом деле спит)
            time.sleep(random.uniform(2,5))
            print (self.name, 'голодный.')
            self.dine()
        else:
            print (self.name,' поел ', self.f_eat_count," раз" )
            return

    def dine(self):
        fork1, fork2 = self.forkOnLeft, self.forkOnRight

        while self.running:
            fork1.acquire(True)
			# простой вариант контроллера__!__#
            #fork2.acquire(True)
            #break
            # не простой вариант контроллера__!__#
            # анти-дедлок -vvvv- (deadlock prevent section)
            locked = fork2.acquire(False)
            if locked: break
            fork1.release()
            # анти-дедлок -^^^^-
            # изменим тактику захвата
            print (self.name, 'поменял порядок захвата.')
            fork1, fork2 = fork2, fork1
            # попробуем начать с другого ресурса
        else:
            return
        self.f_eat_count += 1
        print (self.name, 'начал есть.')
        time.sleep(random.uniform(3,6))
        print (self.name, 'наелся и думает.')
        fork1.release()
        fork2.release()

In [ ]:
def DiningFilosofs(numfilos):
    if (numfilos < 2) or (numfilos > 11) :
       print('Число философов от 2 до 11!')
       return

    # создание семафоров ресурсов (forks) в виде мьютексов (threading.Lock)
    forks = [threading.Lock() for n in range(numfilos)]

    philosophNames = ('1:Кант:','2:Маркс:','3:Платон:','4:Руссо:','5:Сократ:',
                        '6:Пифагор:','7:Гегель:','8:Вольтер:','9:Декарт:',
                        '10:ЛаоЦзы:','11:Рассел:')
    
    # список имен по-английски (а вдруг пригодится?)
    # philosopherNames = ('1:Kant:','2:Marx:','3:Plato:','4:Russo:','5:Sockrat:',
    #                    '6:Pifagor:','7:Gheghel:','8:Voltair:','9:Dekart:',
    #                    '10:Lao:','11:Russel:')

    philosophers = [Filosoff(philosophNames[i], forks[i%numfilos],
                           forks[(i+1)%numfilos]) for i in range(numfilos)]

    random.seed(1537)
    modelstart=time.perf_counter_ns()
    Filosoff.running = True
    for p in philosophers: p.start()
    time.sleep(40)
    Filosoff.running = False
    # Ожидание завершения потоков
    for p in philosophers: p.join()
    modelfini=time.perf_counter_ns()
    print ("== Все закончили! Время =", (modelfini-modelstart)/1000000, "msec")

In [ ]:
DiningFilosofs(10)

Вариант процессов с глобальным счетчиком еды   sushi

Выполнение закончится когда всю еду съедят

In [ ]:
# -*- coding: utf-8 -*-
import threading, random, time

# Обедающие философы

class Filosof(threading.Thread):
    #running = True

    def __init__(self, xname, forkOne, forkTwo):
        super().__init__()
        self.name = xname
        self.forkOnLeft = forkOne
        self.forkOnRight = forkTwo
        self.sushi_eaten =0 

    def run(self) -> None:
        global sushi

        while sushi > 0:
            #  Мудрец думает
            time.sleep(random.normalvariate(4,1))
            print (self.name, 'голодный.')
            # простой вариант !__#
            self.forkOnLeft.acquire(True)
            self.forkOnRight.acquire(True)
            # можно поесть
            if sushi > 0:
               sushi -= 1
               self.sushi_eaten += 1
               print (self.name, 'начал есть.')
            time.sleep(random.expovariate(1/3.0))
            print (self.name, 'наелся и думает.')
            self.forkOnLeft.release()
            self.forkOnRight.release()
        else:
            print (self.name,' поел ', self.sushi_eaten," раз" )
            return
        

In [ ]:
def DiningFilosofas(numfilos):
    if (numfilos < 2) or (numfilos > 11) :
       print('Число философов от 2 до 11!')
       return
    
    # создание семафоров ресурсов (палочек) в виде мьютексов
    forks = [threading.Lock() for n in range(numfilos)]

    philosophNames = ('1:Кант:','2:Маркс:','3:Платон:','4:Руссо:','5:Сократ:',
                        '6:Пифагор:','7:Гегель:','8:Вольтер:','9:Декарт:',
                        '10:ЛаоЦзы:','11:Рассел:')

    philosofs = [Filosof(philosophNames[i], forks[i%numfilos],
                        forks[(i+1)%numfilos]) for i in range(numfilos)]

    random.seed(1537)
    modelstart=time.perf_counter_ns()
    for p in philosofs: p.start()
    # Ожидание завершения потоков
    for p in philosofs: p.join()
    modelfini=time.perf_counter_ns()
    print ("== Все закончили! Время =", (modelfini-modelstart)/1000000, "msec")

In [ ]:
sushi=40
DiningFilosofas(10)

Вариант с официантом Waiter

In [ ]:
import time
from threading import Thread, Lock

sushi = 30

class Waiter:
    def __init__(self) -> None:
        self.mutex = Lock()

    def ask_for_chopsticks(self, left_chopstick: Lock,
                           right_chopstick: Lock) -> None:
        with self.mutex:
            left_chopstick.acquire(True)
            print("Официант слева взял")
            right_chopstick.acquire(True)
            print("Официант справа взял")

    def release_chopsticks(self, left_chopstick: Lock,
                           right_chopstick: Lock) -> None:
        right_chopstick.release()
        print("Официант сложил справа")
        left_chopstick.release()
        print("Официант сложил слева\n")

class Philosoph(Thread):
    def __init__(self, name: str, counter: Waiter,
                left_chopstick: Lock, right_chopstick: Lock):
        super().__init__()
        self.name = name
        self.left_chopstick = left_chopstick
        self.right_chopstick = right_chopstick
        self.waiter = counter

    def run(self) -> None:
        # глобальный счетчик
        global sushi

        while sushi > 0:
            print(f"{self.name} спрашивает официанта")
            self.waiter.ask_for_chopsticks(
                 self.left_chopstick, self.right_chopstick)
            time.sleep(1)
            sushi -= 1
            print(f"{self.name} ест суши. Осталось суши: {sushi}")
            print(f"{self.name} вернул приборы официанту")
            self.waiter.release_chopsticks(
                 self.left_chopstick, self.right_chopstick)
            time.sleep(1)

modelstart=time.perf_counter_ns()
chopstick_a = Lock()
chopstick_b = Lock()
#
waiter=Waiter()
philosopher_1 = Philosoph("Платон",waiter,chopstick_a,chopstick_b)
philosopher_2 = Philosoph("Аристотель",waiter,chopstick_b,chopstick_a)
#
philosopher_1.start()
philosopher_2.start()
#
philosopher_1.join()
philosopher_2.join()
#
modelfini=time.perf_counter_ns()
print ("== Все закончили! Время =", (modelfini-modelstart)/1000000, "msec")

## Вариант конкуренции
Вариант - кто больше съест

In [ ]:
import time
from threading import Thread, Lock

sushi = 1000

class Philosopher(Thread):
    def __init__(self, name: str, left_chopstick: Lock, right_chopstick: Lock):
        super().__init__()
        self.name = name
        self.left_chopstick = left_chopstick
        self.right_chopstick = right_chopstick

    def run(self) -> None:
        # глобальный счетчик
        global sushi
        #
        sushi_eaten = 0
        while sushi > 0:
            self.left_chopstick.acquire(True)
            self.right_chopstick.acquire(True)
            if sushi > 0:
                sushi -= 1
                sushi_eaten += 1
                time.sleep(0.1)
            self.right_chopstick.release()
            self.left_chopstick.release()
            time.sleep(0.1)
        #
        print(f"{self.name} съел {sushi_eaten} суши")

modelstart=time.perf_counter_ns()
chopstick_a = Lock()
chopstick_b = Lock()
#
threads = []
for i in range(10):
    threads.append(Philosopher(f"Философ_{i}", chopstick_a, chopstick_b))
#
for thread in threads:  thread.start()
#
for thread in threads:  thread.join()
#
modelfini=time.perf_counter_ns()
print ("== Все закончили! Время =", (modelfini-modelstart)/1000000, "msec")

В модели однопоточной конкурентности мы избегаем состояний гонки, вызванных неатомарными операциями. В  asyncio есть только один поток, который в каждый момент времени исполняет одну строку кода Python. Это означает, что, даже если операция неатомарна, она все равно будет доведена до конца и другие сопрограммы не смогут прочитать несогласованные данные.

Вместо нескольких потоков модифицировать переменную будут несколько задач.  Блокировки(Lock) помогут гарантировать, что модификации производятся в желаемом синхронизированном порядке.

Главное отличие от lock потока заключается в том, Lock asyncio – объекты, допускающие ожидание, которые приостанавливают выполнение сопрограммы, когда заблокированы. Это значит, что если сопрограмма ожидает освобождения блокировки, то может работать другой код. Кроме того, блокировки asyncio являются 
асинхронными контекстными менеджерами, и предпочтительно использовать их в сочетании с конструкцией async with.

Функция asyncio.gather() дожидается завершения всех допускающих ожидания объектов, прежде чем станет возможен доступ к результатам.

In [ ]:
import asyncio, random
sushi = 40

async def delay(name:str, delay_seconds:int) -> int:
    print(f'{name} будет занят {delay_seconds} с')
    await asyncio.sleep(delay_seconds)
    print(f'{name} закончил есть за {delay_seconds} с')
    return delay_seconds

async def filosof(name:str, lockA: asyncio.Lock, lockB: asyncio.Lock):
    global sushi
    while sushi > 0: 
      print(f'{name} ждет возможности захватить палочки')
      async with lockA:
        async with lockB:
          print(f'{name} начал есть!')
          sushi -= 1
          await delay(name, round(random.uniform(3,6),1))
      print(f'{name} освободил палочки')
      await asyncio.sleep(random.uniform(2,7))

numfils=9
philoNames = ('1:Кант:','2:Маркс:','3:Платон:','4:Руссо:','5:Сократ:',
              '6:Пифагор:','7:Гегель:','8:Вольтер:','9:Декарт:',
              '10:ЛаоЦзы:','11:Рассел:')
modelstart=time.perf_counter_ns()
# создание семафоров ресурсов (forks) в виде 
forks = [asyncio.Lock() for n in range(numfils)]
filosofers = [filosof(philoNames[i], forks[i%numfils], forks[(i+1)%numfils]) for i in range(numfils)]
status = await asyncio.gather( *filosofers)
#
modelfini=time.perf_counter_ns()
print ("== Все закончили! Время =", (modelfini-modelstart)/1000000, "msec")
